# Laser from longitudinal(FROG data) and transverse profile

Following tutorial would be an excellent example of the practical capabalities and ease of use of LASY. In this tutorial we would try to reconstruct a laser pulse for its longitudinal and transverse profile after further denoising of the later.

The tutorial would be proceed with following steps:\
**Step 1:** Load the required packges and define the required parameters.\
**Step 2:** Reconstruct the longitudinal profile.\
**Step 3:** Reconstruct the transverse profile.\
**Step 4:** Combine both the profiles.\
**Step 5:** Denoise the combined profile.\
**Step 6:** Plot both the original and denoised profile.\
**Step 7:** Generate a laser object and export it.


In [ ]:
# For VS code
%matplotlib inline
# For jupyter notbook use "%matplotlib ipympl"

[scikit-image](https://scikit-image.org) is required for the image processing, as the inpute data file for transverse modes is a png. Running the command below will install the package through conda. 

In [ ]:
# %conda install -c conda-forge -y scikit-image

Now lets load all the required packages for the tutorial. Which would include packages for both laser manipulation and to produce a final grapical output.

In [ ]:
# Required LASY Libraries
from lasy.laser import Laser
from lasy.profiles.combined_profile import CombinedLongitudinalTransverseProfile
from lasy.profiles.longitudinal import LongitudinalProfileFromData
from lasy.profiles.transverse import TransverseProfileFromData
from lasy.profiles.transverse.hermite_gaussian_profile import (
    HermiteGaussianTransverseProfile,
)
from lasy.utils.mode_decomposition import hermite_gauss_decomposition

# Standard Libraries
import matplotlib.pyplot as plt
import numpy as np
import skimage
from mpl_toolkits.axes_grid1 import make_axes_locatable

Next, let's define the physical parameters defining our laser pulse. 

In [ ]:
polarization    = (1, 0)        # Linearly polarized in the x direction
energy_J        = 1             # Pulse energy in Joules

## Reconstruct the longitudinal profile

Here as a input file we would be using data recorded using [Frequency-resolved optical gating(FROG)](https://en.wikipedia.org/wiki/Frequency-resolved_optical_gating) techinique. This would provide us with the spectral phase, intensity and frequency(This has been converted to wavelength) which would be vital for our reconstruction.

In [ ]:
# Load the data
data_file = np.loadtxt(
    "https://github.com/user-attachments/files/17042670/df_intensity_spectral.csv",
    delimiter="\t",
)
filename = (
    "https://github.com/user-attachments/files/17042670/df_intensity_spectral.csv"
)

# Extract the required data
E_wavelength = np.loadtxt(filename, usecols=0, dtype="float")   # Wavelength in meters
E_intensity = np.loadtxt(filename, usecols=1, dtype="float")    # Electric field intensity
E_phase = np.loadtxt(filename, usecols=2, dtype="float")        # Electric field phase in radians

Now, lets fininsh the reconstuction by creating a longitudinal profile from all the available data.

In [ ]:
fs = 1e-15                          # Femtoseconds
longitudinal_data = {
    "datatype": "spectral",
    "axis": E_wavelength,           # Make sure that this parameter is monotonically increasing
    "intensity": E_intensity,
    "phase": E_phase,               # Radians
    "dt": fs,                       # User defined tmeporal resolution(only required if datatype is spectral)
}

# Create the longitudinal profile
longitudinal_profile = LongitudinalProfileFromData(
    longitudinal_data, lo=-200 * fs, hi=200 * fs
)

## Reconstruct the transverse profile

For the following reconstruction the data is provided in the .png format, which can be read using scikit image package.(*please uncomment and run the second code block if the package is not installed*)

In [ ]:
# Define the transverse profile of the laser pulse
img             = "https://user-images.githubusercontent.com/27694869/228038930-d6ab03b1-a726-4b41-a378-5f4a83dc3778.png"
transverse_data = skimage.io.imread(img)

# Data cleaning: Remove negative values
intensity_scale = np.max(transverse_data)                     # Determine the maximum value of the intensity within the image/data.
transverse_data[transverse_data < intensity_scale / 100] = 0  # Remove negative values and potential noise.[Recommended: 1% of the maximum value!]


In [ ]:
# Pixel size calibration term
cal             = 0.2e-6                                        # This scale factor comes from a rough estimate.

# Axes
rows, cols      = (transverse_data.shape)                       # Number of rows and columns, as per sensor dimensions
transverse_x_mu = np.linspace(0, cols - 1, cols) * cal
transverse_y_mu = np.linspace(0, rows - 1, rows) * cal
lo              = (transverse_x_mu[0], transverse_y_mu[0])      # Lower bound
hi              = (transverse_x_mu[-1], transverse_y_mu[-1])    # Upper bound

As done earlier for the longitudinal profile, we would construct the transverse profile using extracted data.\
*[This function also centers the data by default]*

In [ ]:
transverse_profile = TransverseProfileFromData(
    transverse_data, [lo[0], lo[1]], [hi[0], hi[1]]
)

Here we would plot the data(uncetered) which we have processed in the above cells. This step is not necessay but acts as a sanity check for us.

In [ ]:
# Plotting the data
fig, ax = plt.subplots()
cax = ax.imshow(
    transverse_data,
    aspect="auto",
    extent=[lo[0], hi[0], lo[1], hi[1]],
)

# Add a colorbar
color_bar = fig.colorbar(cax)

# Add a label to the colorbar
color_bar.set_label(r"Fluence / J/cm$^2$")
ax.set_xlabel(r"x / m")
ax.set_ylabel(r"y / m")
plt.show()

# Combine both the profiles

In [ ]:
central_wavelength  = longitudinal_profile.lambda0  # This is central wavelength of the pulse, calcuated during the longitudinal profile construction

# Original laser profile with the combined longitudinal and transverse profiles
org_laser_profile   = CombinedLongitudinalTransverseProfile(
    wavelength      =longitudinal_profile.lambda0,
    pol             =polarization,
    laser_energy    =energy_J,
    long_profile    =longitudinal_profile,
    trans_profile   =transverse_profile,
)

# Profile denoising
At this stage we would denoise the profiles using Hermite-Gauss modes and combine them afterwards. 
Take a look at the following [example](https://github.com/LASY-org/lasy/blob/13f0e4515493deca36c1375be1d9e83c7e379d42/examples/example_modal_decomposition_data.py).

In [ ]:
# Maximum Hermite-Gauss mode index in x and y (The choise of this parameter is arbitrary)
n_x_max = 2
n_y_max = 2

# Calculate the decomposition and waist of the laser pulse
modeCoeffs, waist = hermite_gauss_decomposition(
    transverse_profile, n_x_max=n_x_max, n_y_max=n_y_max, res=cal
)

*Do check if the estimated value of the waist is sensible. If not, do check if the transverse profile reconstruction is done properly.*

In [ ]:
# Reconstruct the pulse using a series of hermite-gauss modes
energy_frac = 0
for i, mode_key in enumerate(list(modeCoeffs)):
    tmp_transverse_profile = HermiteGaussianTransverseProfile(
        waist, mode_key[0], mode_key[1]
    )
    energy_frac += modeCoeffs[mode_key] ** 2        # Energy fraction of the mode
    if i == 0:                                      # First mode (0,0)
        laser_profile = modeCoeffs[mode_key] * CombinedLongitudinalTransverseProfile(
            central_wavelength,
            polarization,
            energy_J,
            longitudinal_profile,
            tmp_transverse_profile,
        )
    else:                                           # All other modes
        laser_profile += modeCoeffs[mode_key] * CombinedLongitudinalTransverseProfile(
            central_wavelength,
            polarization,
            energy_J,
            longitudinal_profile,
            tmp_transverse_profile,
        )

# Energy loss due to decomposition
energy_loss = 1 - energy_frac
print(f"Energy loss: {energy_loss * 100:.2f}%")

## Plot Denoised Profile

In [ ]:
# Create a grid for plotting
x               = np.linspace(-5 * waist, 5 * waist, 500)
X, Y            = np.meshgrid(x, x)

# Determine the figure parameters
fig, ax         = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)
fig.suptitle(
    "Hermite-Gauss Reconstruction using n_x_max = %i, n_y_max = %i" % (n_x_max, n_y_max)
)

# Plot the original profile
pltextent       = (np.min(x) * 1e6, np.max(x) * 1e6, np.min(x) * 1e6, np.max(x) * 1e6)
prof1           = np.abs(org_laser_profile.evaluate(X, Y, 0)) ** 2
divider0        = make_axes_locatable(ax[0])
ax0_cb          = divider0.append_axes("right", size="5%", pad=0.05)
pl0             = ax[0].imshow(prof1, cmap="magma", extent=pltextent, vmin=0, vmax=np.max(prof1))
cbar0           = fig.colorbar(pl0, cax=ax0_cb)
cbar0.set_label("Intensity (norm.)")
ax[0].set_xlabel("x (micron)")
ax[0].set_ylabel("y (micron)")
ax[0].set_title("Original Profile")

# Plot the reconstructed profile
prof2           = np.abs(laser_profile.evaluate(X, Y, 0)) ** 2
divider1        = make_axes_locatable(ax[1])
ax1_cb          = divider1.append_axes("right", size="5%", pad=0.05)
pl1             = ax[1].imshow(prof2, cmap="magma", extent=pltextent, vmin=0, vmax=np.max(prof1))
cbar1           = fig.colorbar(pl1, cax=ax1_cb)
cbar1.set_label("Intensity (norm.)")
ax[1].set_xlabel("x (micron)")
ax[1].set_ylabel("y (micron)")
ax[1].set_title("Reconstructed Profile")

# Plot the error
prof3           = (prof1 - prof2) / np.max(prof1)  # Normalized error
divider2        = make_axes_locatable(ax[2])
ax2_cb          = divider2.append_axes("right", size="5%", pad=0.05)
pl2             = ax[2].imshow(100 * np.abs(prof3), cmap="magma", extent=pltextent)
cbar2           = fig.colorbar(pl2, cax=ax2_cb)
cbar2.set_label("|Intensity Error| (%)")
ax[2].set_xlabel("x (micron)")
ax[2].set_ylabel("y (micron)")
ax[2].set_title("Error")

plt.show()

# Create a laser

Now we have done the hard part!\
The only thing that remains now is to create a laser object and export it, to then plug it into any simulation program of your choice!!!

In [ ]:
# Create the laser object
dimensions  = "rt"                  # Use cylindrical geometry
lo          = (0,    -50e-15)       # Lower bounds of the simulation box
hi          = (40e-6, 50e-15)       # Upper bounds of the simulation box
num_points  = (500, 400)            # Number of points in each dimension

laser = Laser(dimensions, lo, hi, num_points, laser_profile)    # Laser
laser.normalize(energy_J * energy_frac)                         # Normalize the laser energy
laser.show()

# Save the laser object to a file
laser.write_to_file("Laser_rt_denoised","h5",save_as_vector_potential=True)

*Constructing the object using 3D geometry, might take a while to run depending on the hardware you are using.*

In [ ]:
dimensions = "xyt"  # Use 3D geometry
lo              = (-40e-6, -20e-6, -50e-15)     # Lower bounds of the simulation box
hi              = (40e-6,   20e-6,  50e-15)        # Upper bounds of the simulation box
num_points      = (300, 300, 200)               # Number of points in each dimension
# num_points    = (50, 50, 20)                  # Low res for quick tests

laser_xyt       = Laser(dimensions, lo, hi, num_points, laser_profile)
laser_xyt.normalize(energy_J * energy_frac)
laser_xyt.show()

# Save the laser object to a file
laser_xyt.write_to_file("Laser_xyt_denoised","h5",save_as_vector_potential=True)